In [ ]:
# IMPORTANT: SOME KAGGLE DATA SOURCES ARE PRIVATE
# RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES.
import kagglehub
kagglehub.login()


In [ ]:
# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.

state_farm_distracted_driver_detection_path = kagglehub.competition_download('state-farm-distracted-driver-detection')

print('Data source import complete.')


In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All"
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/state-farm-distracted-driver-detection/sample_submission.csv
/kaggle/input/state-farm-distracted-driver-detection/driver_imgs_list.csv


In [ ]:
# =========================================
# 0) Kaggle 환경 준비
# =========================================
import os, warnings
warnings.filterwarnings("ignore")

BASE_DIR   = "/kaggle/input/state-farm-distracted-driver-detection"
DRIVER_CSV = f"{BASE_DIR}/driver_imgs_list.csv"
TRAIN_DIR  = f"{BASE_DIR}/imgs/train"
TEST_DIR   = f"{BASE_DIR}/imgs/test"

print("📁 BASE_DIR:", BASE_DIR)
print("📄 driver_imgs_list.csv:", os.path.exists(DRIVER_CSV))
print("📁 Train:", os.path.exists(TRAIN_DIR))
print("📁 Test :", os.path.exists(TEST_DIR))

# =========================================
# 1) 라이브러리
# =========================================
import math
import numpy as np
import pandas as pd
from PIL import Image, ImageOps, ImageEnhance

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, models

from sklearn.model_selection import KFold
from tqdm import tqdm

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("DEVICE:", device)

# =========================================
# 2) 설정값
# =========================================
IMG_SIZE      = 224
NUM_CLASSES   = 10
BATCH_SIZE    = 64
EPOCHS        = 40
LR            = 5e-5
WEIGHT_DECAY  = 1e-4
NUM_WORKERS   = 2
PATIENCE      = 6
FOLD_INDEX    = 1

OFFLINE_CACHE_DIR = "/kaggle/working/statefarm_offline_cache"
os.makedirs(OFFLINE_CACHE_DIR, exist_ok=True)

# =========================================
# 3) 데이터 로드 + KFold
# =========================================
driver_df = pd.read_csv(DRIVER_CSV)
ALL_DRIVERS = sorted(driver_df["subject"].unique())
kfold = KFold(n_splits=5, shuffle=True, random_state=42)

# =========================================
# 4) OFFLINE 팀 공통 전처리
# =========================================
def team_processing_offline_pil(img_pil, out_size=IMG_SIZE):
    img = img_pil.convert("RGB")
    img = img.resize((out_size, out_size), Image.BILINEAR)

    img = ImageEnhance.Contrast(img).enhance(1.05)
    img = ImageEnhance.Sharpness(img).enhance(1.05)

    r, g, b = img.split()
    r, g, b = ImageOps.equalize(r), ImageOps.equalize(g), ImageOps.equalize(b)
    return Image.merge("RGB", (r, g, b))


def build_offline_cache(train_dir, driver_df, driver_list, cache_root):
    for c in [f"c{i}" for i in range(NUM_CLASSES)]:
        os.makedirs(os.path.join(cache_root, c), exist_ok=True)

    subset = driver_df[driver_df["subject"].isin(driver_list)]
    for row in tqdm(subset.itertuples(), total=len(subset), desc="OFFLINE caching"):
        cls, img = row.classname, row.img
        src = os.path.join(train_dir, cls, img)
        dst = os.path.join(cache_root, cls, img)

        if os.path.exists(dst):
            continue

        try:
            with Image.open(src) as im:
                out = team_processing_offline_pil(im)
                out.save(dst, quality=95)
        except:
            pass

# =========================================
# 5) ONLINE transform
# =========================================
normalize_imagenet = transforms.Normalize([0.485,0.456,0.406],
                                          [0.229,0.224,0.225])

transform_online_train = transforms.Compose([
    transforms.RandomResizedCrop(IMG_SIZE, scale=(0.8, 1.0)),
    transforms.RandomHorizontalFlip(),
    transforms.ColorJitter(0.15, 0.15, 0.10),
    transforms.RandomRotation(12),
    transforms.ToTensor(),
    normalize_imagenet
])

# 🔥 Option A — CenterCrop 추가
transform_eval = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    normalize_imagenet
])

# =========================================
# 6) Dataset 클래스
# =========================================
class DriverDataset(Dataset):
    def __init__(self, data_dir, driver_df, driver_list, transform=None):
        self.transform = transform
        self.images, self.labels = [], []

        subset = driver_df[driver_df["subject"].isin(driver_list)]
        for _, row in subset.iterrows():
            cls = row["classname"]
            img = row["img"]
            self.images.append(os.path.join(data_dir, cls, img))
            self.labels.append(int(cls[1:]))

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        path = self.images[idx]
        label = self.labels[idx]
        with Image.open(path) as im:
            im = im.convert("RGB")
            if self.transform:
                im = self.transform(im)
        return im, label

# =========================================
# 7) Conv2d WS + GN 적용
# =========================================
class Conv2dWS(nn.Conv2d):
    def forward(self, x):
        w = self.weight
        w = w - w.mean(dim=(1,2,3), keepdim=True)
        s = w.view(w.size(0), -1).std(dim=1, keepdim=True).view(-1,1,1,1) + 1e-5
        w = w / s
        return nn.functional.conv2d(x, w, self.bias, self.stride,
                                    self.padding, self.dilation, self.groups)

def replace_bn_with_gn(module, groups=32):
    for name, m in list(module.named_children()):
        if isinstance(m, nn.BatchNorm2d):
            setattr(module, name, nn.GroupNorm(groups, m.num_features))
        else:
            replace_bn_with_gn(m, groups)

def replace_conv_with_ws(module):
    for name, m in list(module.named_children()):
        if isinstance(m, nn.Conv2d):
            ws = Conv2dWS(m.in_channels, m.out_channels,
                          m.kernel_size, m.stride, m.padding,
                          m.dilation, m.groups, bias=(m.bias is not None))
            ws.weight.data.copy_(m.weight.data)
            if m.bias is not None: ws.bias.data.copy_(m.bias.data)
            setattr(module, name, ws)
        else:
            replace_conv_with_ws(m)

# =========================================
# 8) ResNet34(GN+WS) + Dropout0.1 + layer4 only train
# =========================================
def build_resnet34_gn_ws_dropout(num_classes=NUM_CLASSES):
    model = models.resnet34(weights=models.ResNet34_Weights.IMAGENET1K_V1)

    in_feat = model.fc.in_features
    model.fc = nn.Sequential(
        nn.Dropout(0.1),
        nn.Linear(in_feat, num_classes)
    )

    replace_bn_with_gn(model.layer3)
    replace_bn_with_gn(model.layer4)
    replace_conv_with_ws(model.layer3)
    replace_conv_with_ws(model.layer4)

    for p in model.layer1.parameters(): p.requires_grad = False
    for p in model.layer2.parameters(): p.requires_grad = False
    for p in model.layer3.parameters(): p.requires_grad = False
    for p in model.layer4.parameters(): p.requires_grad = True

    return model


# =========================================
# 9) Train/Eval 함수
# =========================================
def train_one_epoch(model, loader, criterion, optimizer):
    model.train()
    total_loss, correct, total = 0, 0, 0
    for x, y in tqdm(loader, desc="Train", leave=False):
        x, y = x.to(device), y.to(device)
        optimizer.zero_grad(set_to_none=True)
        out = model(x)
        loss = criterion(out, y)
        loss.backward()
        optimizer.step()

        total_loss += loss.item()*x.size(0)
        correct    += (out.argmax(1)==y).sum().item()
        total      += y.size(0)
    return total_loss/total, correct/total*100


@torch.no_grad()
def evaluate(model, loader, criterion):
    model.eval()
    total_loss, correct, total = 0, 0, 0
    for x, y in tqdm(loader, desc="Valid", leave=False):
        x, y = x.to(device), y.to(device)
        out = model(x)
        loss = criterion(out, y)

        total_loss += loss.item()*x.size(0)
        correct    += (out.argmax(1)==y).sum().item()
        total      += y.size(0)
    return total_loss/total, correct/total*100


class EarlyStopping:
    def __init__(self, patience=6, delta=0.0):
        self.patience = patience
        self.delta = delta
        self.best = None
        self.count = 0
        self.stop = False

    def step(self, metric, model, path):
        if self.best is None or (self.best - metric) > self.delta:
            self.best = metric
            self.count = 0
            torch.save(model.state_dict(), path)
            print(f"  🔥 Best 갱신 → {metric:.4f}")
        else:
            self.count += 1
            print(f"  ⏳ EarlyStopping {self.count}/{self.patience}")
            if self.count >= self.patience:
                self.stop = True


# =========================================
# 🔥 10) Fold split & Cache build
# =========================================
folds = list(kfold.split(ALL_DRIVERS))
train_idx, val_idx = folds[FOLD_INDEX]

train_drivers = [ALL_DRIVERS[i] for i in train_idx]
val_drivers   = [ALL_DRIVERS[i] for i in val_idx]

print("TRAIN:", len(train_drivers), " VAL:", len(val_drivers))

build_offline_cache(TRAIN_DIR, driver_df, train_drivers, OFFLINE_CACHE_DIR)
build_offline_cache(TRAIN_DIR, driver_df, val_drivers,   OFFLINE_CACHE_DIR)

train_ds = DriverDataset(OFFLINE_CACHE_DIR, driver_df, train_drivers, transform=transform_online_train)
val_ds   = DriverDataset(OFFLINE_CACHE_DIR, driver_df, val_drivers,   transform=transform_eval)

train_loader = DataLoader(train_ds, batch_size=BATCH_SIZE, shuffle=True, num_workers=NUM_WORKERS)
val_loader   = DataLoader(val_ds,   batch_size=BATCH_SIZE, shuffle=False, num_workers=NUM_WORKERS)


# =========================================
# 🔥 11) 모델 + Optim + Scheduler
# =========================================
model = build_resnet34_gn_ws_dropout().to(device)

optim_params = [p for p in model.parameters() if p.requires_grad]
optimizer = optim.AdamW(optim_params, lr=LR, weight_decay=WEIGHT_DECAY)
criterion = nn.CrossEntropyLoss()

warmup_epochs = 3
def lr_lambda(epoch):
    if epoch < warmup_epochs:
        return float(epoch+1)/warmup_epochs
    prog = (epoch-warmup_epochs)/(EPOCHS-warmup_epochs)
    return 0.5*(1+math.cos(math.pi*prog))

scheduler = optim.lr_scheduler.LambdaLR(optimizer, lr_lambda)

print("Trainable params:", sum(p.numel() for p in optim_params))


# =========================================
# 🔥 12) 학습 루프
# =========================================
history = {"train_loss":[], "train_acc":[], "val_loss":[], "val_acc":[]}
best_model_path = "/kaggle/working/resnet34_gnws_best.pth"
es = EarlyStopping(patience=PATIENCE)

for epoch in range(EPOCHS):
    print(f"\n🔵 Epoch {epoch+1}/{EPOCHS} | LR={optimizer.param_groups[0]['lr']:.6f}")

    t_loss, t_acc = train_one_epoch(model, train_loader, criterion, optimizer)
    v_loss, v_acc = evaluate(model, val_loader, criterion)

    history["train_loss"].append(t_loss)
    history["train_acc"].append(t_acc)
    history["val_loss"].append(v_loss)
    history["val_acc"].append(v_acc)

    print(f"  Train Loss={t_loss:.4f}, Acc={t_acc:.2f}%")
    print(f"  Val   Loss={v_loss:.4f}, Acc={v_acc:.2f}%")

    es.step(v_loss, model, best_model_path)
    if es.stop:
        print("🛑 EarlyStopping 발동")
        break

    scheduler.step()


# =========================================
# 🔥 13) 테스트 세트 추론 + 제출 파일
# =========================================
best_path = best_model_path
model.load_state_dict(torch.load(best_path, map_location=device))
model.eval()

test_imgs = sorted(os.listdir(TEST_DIR))

class TestSet(Dataset):
    def __len__(self): return len(test_imgs)
    def __getitem__(self, i):
        p = os.path.join(TEST_DIR, test_imgs[i])
        with Image.open(p) as im:
            im = im.convert("RGB")
            im = transform_eval(im)
        return im, test_imgs[i]

test_loader = DataLoader(TestSet(), batch_size=BATCH_SIZE, shuffle=False)

preds, names = [], []
with torch.no_grad():
    for x, fn in tqdm(test_loader, desc="Predict"):
        x = x.to(device)
        out = model(x)
        prob = torch.softmax(out, dim=1).cpu().numpy()
        preds.append(prob)
        names += fn

preds = np.vstack(preds)
df = pd.DataFrame(preds, columns=[f"c{i}" for i in range(NUM_CLASSES)])
df.insert(0, "img", names)

SUBMIT_PATH = "/kaggle/working/submission.csv"
df.to_csv(SUBMIT_PATH, index=False)

print("저장 완료:", SUBMIT_PATH)
df.head()


📁 BASE_DIR: /kaggle/input/state-farm-distracted-driver-detection
📄 driver_imgs_list.csv: True
📁 Train: True
📁 Test : True
DEVICE: cuda
TRAIN: 21  VAL: 5


OFFLINE caching: 100%|██████████| 4006/4006 [01:01<00:00, 65.35it/s]
Downloading: "https://download.pytorch.org/models/resnet34-b627a593.pth" to /root/.cache/torch/hub/checkpoints/resnet34-b627a593.pth
100%|██████████| 83.3M/83.3M [00:00<00:00, 165MB/s] 


Trainable params: 13129034

🔵 Epoch 1/40 | LR=0.000017


  Train Loss=1.3074, Acc=60.44%
  Val   Loss=1.1638, Acc=63.35%
  🔥 Best 갱신 → 1.1638

🔵 Epoch 2/40 | LR=0.000033


  Train Loss=0.3264, Acc=91.58%
  Val   Loss=0.8970, Acc=74.69%
  🔥 Best 갱신 → 0.8970

🔵 Epoch 3/40 | LR=0.000050


...

  Train Loss=0.0395, Acc=98.85%
  Val   Loss=1.0303, Acc=73.79%
  ⏳ EarlyStopping 5/6

🔵 Epoch 9/40 | LR=0.000048


  Train Loss=0.0355, Acc=98.93%
  Val   Loss=1.1049, Acc=74.91%
  ⏳ EarlyStopping 6/6
🛑 EarlyStopping 발동


Predict: 100%|██████████| 1246/1246 [18:51<00:00,  1.10it/s]


저장 완료: /kaggle/working/submission.csv


,img,c0,c1,c2,c3,c4,c5,c6,c7,c8,c9
0,img_1.jpg,0.002757,0.847939,0.014127,0.036086,0.001014,0.037280,0.001430,0.050067,0.006261,0.003039
1,img_10.jpg,0.002469,0.001174,0.000045,0.000817,0.001270,0.989551,0.002906,0.000682,0.000618,0.000469
2,img_100.jpg,0.789620,0.015487,0.068741,0.004313,0.000458,0.028160,0.005918,0.012718,0.034943,0.039641
3,img_1000.jpg,0.001085,0.014567,0.971250,0.000048,0.000072,0.001027,0.001597,0.000172,0.009776,0.000407
4,img_100000.jpg,0.010779,0.005759,0.000138,0.133363,0.001521,0.836966,0.001114,0.000602,0.005989,0.003770
